In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from mysql.connector import Error
from datetime import datetime, date, timedelta
import re
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Definimos la ruta en donde queremos guardar los archivos
ruta_guardar = '/home/estadistico/Documents/Erick/Reportes diarios'
ruta_pagos = '/home/estadistico/Documents/Erick'
mes = 20201201
mes2 = 'Diciembre2020'
mes3 = pd.to_datetime('01/12/2020',dayfirst=True)
prim_sem_mes = 49

In [3]:
#Agregamos las variables a ocupar
servidor = '192.168.15.12'
puerto = int('3306')
usuario = 'estadisticas'
contrasena = 'estadisticas8474'
base = 'procesos_externos'

In [4]:
#Lectura pagos Credifiel
sheet_cred = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Pagos/')
pagos_cred = pd.DataFrame()
for i in sheet_cred:
    pagos_cred_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Pagos/'+i+'')
    pagos_cred = pd.concat([pagos_cred,pagos_cred_new],ignore_index=True)
pagos_cred = pagos_cred.iloc[:,0:5]
pagos_cred2 = pd.pivot_table(pagos_cred,index=['ID','FECHA'],values=['IMPORTE'],aggfunc=np.sum)
pagos_cred2 = pd.DataFrame(pagos_cred2.to_records())

In [5]:
#Lectura asignacion Credifiel
sheet_cred2 = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Asignacion/')
asignacion_cred = pd.DataFrame()
for j in sheet_cred2:
    asignacion_cred_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Asignacion/'+j+'')
    asignacion_cred = pd.concat([asignacion_cred,asignacion_cred_new],ignore_index=True)
asignacion_cred = asignacion_cred.loc[:,['CREDITO','ARANCEL']]
#asignacion_cred2 = pd.pivot_table(asignacion_cred,index=['ID','FECHA'],values=['IMPORTE'],aggfunc=np.sum)
#asignacion_cred2 = pd.DataFrame(asignacion_cred2.to_records())

In [6]:
#Asignamos valores a los parametros \n"
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y %H:%M')
hoy = np.datetime64('today')
ayer = np.datetime64(date.today() - timedelta(1))
today2 = date.today().strftime('%d-%m-%Y')

In [7]:
#Hacemos la conexion con el servidor\n",
try: 
    conn = mysql.connector.connect(user=usuario,
                               password=contrasena,
                               host=servidor,
                               port=3306,
                               database=base)
    conn.set_charset_collation('latin1')
except mysql.Error as e:
    print("Failed to execute stored procedure: {}".format(error))

cursor = conn.cursor()

In [8]:
#Funcion para hacer la consulta sql, ya sea de gestion o de asignacion en donde requerimos tres parametros, la tabla de donde vamos a sacar la consulta, en caso de asignacin requerimos los otros dos parametros fecha inicial y fecha final
def consulta_sql(table,from_date=None,to_date=None,campana=None):
    if campana is None:
        if from_date is None:
            sql_mes = cursor.callproc(table,)
            for result in cursor.stored_results():
                sql_mes = pd.DataFrame(result.fetchall())
        else:
            sql_mes = cursor.callproc(table, [from_date,to_date])
            for result in cursor.stored_results():
                sql_mes = pd.DataFrame(result.fetchall())
    else:
        sql_mes = cursor.callproc(table, [from_date,to_date,campana])
        for result in cursor.stored_results():
                sql_mes = pd.DataFrame(result.fetchall())
    sql_mes = sql_mes.drop_duplicates()
    return sql_mes

In [9]:
#Funcion para transformar varias columnas a numero
def transform_numeric(df,cols):
    return df[cols].apply(pd.to_numeric, errors='coerce')

#Funcion que convierte a dataframe y rellena espacios con 0
def to_dataframe(df):
    df = pd.DataFrame(df.to_records())
    df.fillna(0,inplace=True)
    return df

#Funcion que quita el % del string
def percent_tonumeric(df,cols):
    for c in cols:
        df[c] = [x[:-2] for x in df[c]]
    return transform_numeric(df,cols)

# Performance Credifiel

In [10]:
#Hacemos la consulta referente al performance del Gestor
performance_all = consulta_sql('procesos_externos.all_rpt_agent_performance',20200101,today)
performance_all.columns = ['fecha','usuario','nombre_usuario','user_group','llamadas','aht','ocupacion','adherencia','login_vici','logout_vici','staff_time','wait_time','talk_time','dispo_time','pause','login','manual','gestion','agua','alimentos','sanitario','retroalimentacion','tranferencia','validacion','whatsapp','incumplimiento','recordatorios','capacitacion','sistemas','lagged','tipificando','*']

In [11]:
performance_cred = performance_all.loc[performance_all['user_group']=='AFINES']
performance_cred[['ocupacion','adherencia']] = percent_tonumeric(performance_cred,['ocupacion','adherencia'])

In [12]:
performance_cred['hora_login'] = [int(x[:2]) for x in performance_cred['login_vici']]
performance_cred['hora_logout'] = [int(x[:2]) for x in performance_cred['logout_vici']]
performance_cred['turno'] = np.where((performance_cred['hora_logout']<=14)&(performance_cred['hora_logout']-performance_cred['hora_login']<=8),'M',
                                    np.where((performance_cred['hora_logout']>17)&(performance_cred['hora_logout']-performance_cred['hora_login']<=8),'V','Mixto'))

In [13]:
performance_cred['horas_trabajadas'] = performance_cred['hora_logout']-performance_cred['hora_login']
performance_cred['llamadas_hora'] = performance_cred['llamadas']/performance_cred['horas_trabajadas']
performance_cred['llamadas_hora'] = performance_cred['llamadas_hora'].round(1)
performance_cred.head(1)

,fecha,usuario,nombre_usuario,user_group,llamadas,aht,ocupacion,adherencia,login_vici,logout_vici,...,capacitacion,sistemas,lagged,tipificando,*,hora_login,hora_logout,turno,horas_trabajadas,llamadas_hora
1329,2020-10-13,GONMORALESBM,GONZALEZ MORALES BRENDA MARLEN,AFINES,312,00:00:56,40.54,72.42,07:09:34,16:50:10,...,00:00:00,00:00:00,00:00:00,00:07:52,00:03:45,7,16,Mixto,9,34.7


In [38]:
control_performance_cred_turno = pd.pivot_table(performance_cred,index=['turno'],values=['llamadas','ocupacion','horas_trabajadas'],aggfunc=[np.sum,np.mean,'count',min,max])
control_performance_cred_turno = to_dataframe(control_performance_cred_turno)
#control_performance_cred_turno = control_performance_cred_turno.iloc[:,[0,1,2,3,4,5,6,7,8,11,12,13,14,15,16]]
#control_performance_cred_turno.columns = ['Gestor','Turno','SumLlamadasDia','SumLlamadasHora','SumOcupacion','PromedioLlamadasDia','PromedioLlamadasHora','PromedioOcupacion','DiasAsitencia','MinLlamadasDia','MinLlamadasHora','MinOcupacion','MaxLlamadasDia','MaxLlamadasHora','MaxOcupacion']
control_performance_cred_turno

,turno,"('sum', 'horas_trabajadas')","('sum', 'llamadas')","('sum', 'ocupacion')","('mean', 'horas_trabajadas')","('mean', 'llamadas')","('mean', 'ocupacion')","('count', 'horas_trabajadas')","('count', 'llamadas')","('count', 'ocupacion')","('min', 'horas_trabajadas')","('min', 'llamadas')","('min', 'ocupacion')","('max', 'horas_trabajadas')","('max', 'llamadas')","('max', 'ocupacion')"
0,M,825,24136,5172.13,5.892857,172.400000,36.943786,140,140,140,0,0,0.00,8,654,60.21
1,Mixto,1005,33411,3143.76,11.686047,388.500000,36.555349,86,86,86,1,31,13.79,14,1057,57.35
2,V,254,7340,1452.48,6.512821,188.205128,37.243077,39,39,39,5,79,14.67,8,593,73.74


In [40]:
control_performance_cred = pd.pivot_table(performance_cred,index=['nombre_usuario'],values=['llamadas','ocupacion','horas_trabajadas'],aggfunc=[np.sum,np.mean,'count',min,max])
control_performance_cred = to_dataframe(control_performance_cred)
#control_performance_cred = control_performance_cred.iloc[:,[0,1,2,6,7,8]]
#control_performance_cred.columns = ['Gestor','SumHorasTrabajadas','SumLlamadasDia','PromedioLlamadasDia','PromedioLlamadasHora','PromedioOcupacion']
control_performance_cred.head(3)

,nombre_usuario,"('sum', 'horas_trabajadas')","('sum', 'llamadas')","('sum', 'ocupacion')","('mean', 'horas_trabajadas')","('mean', 'llamadas')","('mean', 'ocupacion')","('count', 'horas_trabajadas')","('count', 'llamadas')","('count', 'ocupacion')","('min', 'horas_trabajadas')","('min', 'llamadas')","('min', 'ocupacion')","('max', 'horas_trabajadas')","('max', 'llamadas')","('max', 'ocupacion')"
0,AGUILAR RIOS DANIEL ALEJANDRO,30,1580,278.35,5.000000,263.333333,46.391667,6,6,6,2,81,34.48,7,377,58.45
1,GARCIA CHAVEZ BRAYAN,240,3416,819.79,8.571429,122.000000,29.278214,28,28,28,1,14,14.81,14,306,45.55
2,GONZALEZ MORALES BRENDA MARLEN,334,7470,1384.24,8.146341,182.195122,33.761951,41,41,41,5,31,13.79,13,607,54.07


In [42]:
control_performance_cred2 = control_performance_cred[np.abs(control_performance_cred - 
control_performance_cred.mean()) <= (3 * control_performance_cred.std())]
#cols = ['']
control_performance_cred2 = control_performance_cred2.replace({'0':np.nan, 0:np.nan})
control_performance_cred2.loc['mean'] = control_performance_cred2.mean().round(2)

In [36]:
col = ['Gestor', 'Turno', 'SumLlamadasDia', 'SumLlamadasHora', 'SumOcupacion',
       'PromedioLlamadasDia', 'PromedioLlamadasHora', 'PromedioOcupacion',
       'DiasAsitencia', 'MinLlamadasDia', 'MinLlamadasHora', 'MinOcupacion',
       'MaxLlamadasDia', 'MaxLlamadasHora', 'MaxOcupacion']
control_performance_cred_turno3 = control_performance_cred_turno[np.abs(control_performance_cred_turno - 
control_performance_cred_turno.mean()) <= (3 * control_performance_cred_turno.std())]
#cols = ['']
control_performance_cred_turno3 = control_performance_cred_turno3.replace({'0':np.nan, 0:np.nan})

In [37]:
control_performance_cred_turno3.loc['mean'] = control_performance_cred_turno3.mean().round(2)
control_performance_cred_turno3

,turno,"('sum', 'horas_trabajadas')","('sum', 'llamadas')","('sum', 'ocupacion')","('mean', 'horas_trabajadas')","('mean', 'llamadas')","('mean', 'ocupacion')","('count', 'horas_trabajadas')","('count', 'llamadas')","('count', 'ocupacion')","('min', 'horas_trabajadas')","('min', 'llamadas')","('min', 'ocupacion')","('max', 'horas_trabajadas')","('max', 'llamadas')","('max', 'ocupacion')"
0,NaN,825.0,24136.0,5172.13,5.892857,172.400000,36.943786,140.0,140.0,140.0,NaN,NaN,NaN,8.0,654.00,60.21
1,NaN,1005.0,33411.0,3143.76,11.686047,388.500000,36.555349,86.0,86.0,86.0,1.0,31.0,13.79,14.0,1057.00,57.35
2,NaN,254.0,7340.0,1452.48,6.512821,188.205128,37.243077,39.0,39.0,39.0,5.0,79.0,14.67,8.0,593.00,73.74
3,NaN,2084.0,64887.0,9768.37,7.864151,244.856604,36.861774,265.0,265.0,265.0,NaN,NaN,NaN,14.0,1057.00,73.74
mean,NaN,1042.0,32443.5,4884.18,7.990000,248.490000,36.900000,132.5,132.5,132.5,3.0,55.0,14.23,11.0,840.25,66.26


# Gestion detallado Credifiel

In [19]:
#Inicio y fin de meses que nos interesa conocer
inicio_mes = [20200101,20200201,20200301,20200401,20200501,20200601,20200701,20200801,20200901,20201001,20201101]
fin_mes = [20200131,20200228,20200331,20200430,20200531,20200630,20200731,20200831,20200930,20201031,20201130]

In [20]:
#Lectura de cada uno de los meses
gestion_cred = pd.DataFrame()
for i,j in zip(inicio_mes,fin_mes):
    gestion_cred_mes = consulta_sql('credifiel_rpt_gestiones_detallado',i,j)
    gestion_cred = pd.concat([gestion_cred,gestion_cred_mes],ignore_index=True)
    print(i)

20200101
20200201
20200301
20200401
20200501
20200601
20200701
20200801
20200901
20201001
20201101


In [21]:
#Hacemos la consulta referente a las gestiones de Credifiel
gestion_cred.columns = ['folio_gestion','unegocio_id','fecha_gestion','hora_gestion','credito','nombre_credito','telefono','usuario','nombre_usuario','accion','resultado','fecha_promesa','monto_promesa','comentarios','accion_credifiel','resultado_credifiel','dictamen']

In [22]:
#Hacemos el consolidado de la asignacion y promesado
promesas_cred = gestion_cred.loc[gestion_cred['accion_credifiel']=='PDP']
cols = ['monto_promesa']
promesas_cred[cols] = promesas_cred[cols].apply(pd.to_numeric, errors='coerce')
promesas_cred = promesas_cred.loc[promesas_cred['monto_promesa']>50]
promesas_cred = promesas_cred[(promesas_cred['monto_promesa']<500000)]
promesas_cred['credito'] = [x[3:] for x in promesas_cred['credito']]
promesas_cred['credito'] = pd.to_numeric(promesas_cred['credito'])
promesas_cred.head(3)

,folio_gestion,unegocio_id,fecha_gestion,hora_gestion,credito,nombre_credito,telefono,usuario,nombre_usuario,accion,resultado,fecha_promesa,monto_promesa,comentarios,accion_credifiel,resultado_credifiel,dictamen
245,6506980,1,2020-01-20,0 days 09:51:48,292725,OVANDO MORALES PASCUAL,9681246856,MARIVERAAN,MARTINEZ RIVERA ANDREA,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-01-22,5000.0,9681246856 /P/ TITULAR ESTARA REALIZANDO UN PA...,PDP,Promesa de Pago,CONTACTO
436,6545399,1,2020-01-24,0 days 11:29:00,273404,SANCHEZ ALVAREZ DIANA,7471007942,CRPEREZAS,ALEJANDRA SARAI DE LA CRUZ PÃ¨REZ,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-01-24,700.0,7471007942/IN/ TT REALIZARA SU PAGO EN PARCIAL...,PDP,Promesa de Pago,CONTACTO
758,6560210,1,2020-01-27,0 days 07:35:53,273404,SANCHEZ ALVAREZ DIANA,7471007942,CRPEREZAS,ALEJANDRA SARAI DE LA CRUZ PÃ¨REZ,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-01-31,1000.0,7471007942/M/ TT REALIZARA SU SEGUNDO PAGO EL...,PDP,Promesa de Pago,CONTACTO


In [23]:
pagos_cred_fact = pd.merge(pagos_cred2,asignacion_cred,how='left',left_on=['ID'],right_on=['CREDITO'])
pagos_cred_fact['ARANCEL'] = np.where(pagos_cred_fact['ARANCEL'].isna(),np.mean(pagos_cred_fact['ARANCEL']),pagos_cred_fact['ARANCEL'])

In [24]:
consolidado_pagos_cred = pd.merge(promesas_cred,pagos_cred_fact,how='left',left_on=['credito'],right_on=['ID'])
try: consolidado_pagos_cred['fecha_promesa'] = pd.to_datetime(consolidado_pagos_cred['fecha_promesa'],dayfirst=False)
except: consolidado_pagos_cred['fecha_promesa'] = consolidado_pagos_cred['fecha_gestion']

In [25]:
#Condicionamos a los gestores cuyas promesas y pagos estan en un plazo de 3 dias
#consolidado_pagos_cred['PagoUnico'] = [np.where((z < x+pd.Timedelta(4, unit='D')) & ((z > x-pd.Timedelta(4, unit='D') )), y, 0) for x,y,z in zip(consolidado_pagos_cred['fecha_promesa'],consolidado_pagos_cred['IMPORTE'],consolidado_pagos_cred['FECHA'])]
#consolidado_pagos_cred['PagoUnico'] = [x.item() for x in consolidado_pagos_cred['PagoUnico']]

In [26]:
consolidado_pagos_cred['PagoReal'] = [x*y for x,y in zip(consolidado_pagos_cred['IMPORTE'],consolidado_pagos_cred['ARANCEL'])]
consolidado_pagos_cred2 = consolidado_pagos_cred.drop_duplicates(subset=['credito','IMPORTE','FECHA','usuario'])
consolidado_pagos_cred2['mes'] = pd.DatetimeIndex(consolidado_pagos_cred2['fecha_promesa']).month

In [27]:
gestor_mes = pd.pivot_table(consolidado_pagos_cred2,index=['usuario'],values=['monto_promesa'],columns=['mes'],aggfunc=np.sum)
gestor_mes = pd.DataFrame(gestor_mes.to_records())
gestor_mes# = gestor_mes[gestor_mes.isnull().sum(axis=1) < 8]

,usuario,"('monto_promesa', 1)","('monto_promesa', 2)","('monto_promesa', 3)","('monto_promesa', 4)","('monto_promesa', 5)","('monto_promesa', 6)","('monto_promesa', 7)","('monto_promesa', 8)","('monto_promesa', 9)","('monto_promesa', 10)","('monto_promesa', 11)"
0,CRPEREZAS,700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DOMINGUEZHEANIT,NaN,NaN,6000.0,4060.5,32760.0,NaN,NaN,NaN,NaN,NaN,NaN
2,ENRIQUEZLI,NaN,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ESQUIVELESCYEL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,450.79,7727.42,NaN,NaN
4,GACHAVEZBR,NaN,NaN,NaN,NaN,NaN,34848.0,180267.97,105623.29,58076.62,13000.00,NaN
5,GONMORALESBM,NaN,NaN,24000.0,22600.0,7593.0,99305.0,125616.42,85019.22,72305.20,137182.55,52514.59
6,HEGONZALEZSA,NaN,NaN,3000.0,8000.0,6780.0,1500.0,NaN,NaN,NaN,NaN,NaN
7,HERNANDEZGARO,NaN,NaN,NaN,NaN,NaN,59513.0,79655.00,54673.13,56079.99,222056.54,93094.83
8,LOVAERE,NaN,NaN,NaN,NaN,4987.0,46538.0,NaN,NaN,NaN,NaN,NaN
9,MARIVERAAN,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
gestor_mes2 = gestor_mes[np.abs(gestor_mes - 
gestor_mes.mean()) <= (3 * gestor_mes.std())]
#cols = ['']
gestor_mes2 = gestor_mes2.replace({'0':np.nan, 0:np.nan})

In [29]:
gestor_mes2.loc['mean'] = gestor_mes2.mean().round(2)
gestor_mes2

,usuario,"('monto_promesa', 1)","('monto_promesa', 2)","('monto_promesa', 3)","('monto_promesa', 4)","('monto_promesa', 5)","('monto_promesa', 6)","('monto_promesa', 7)","('monto_promesa', 8)","('monto_promesa', 9)","('monto_promesa', 10)","('monto_promesa', 11)"
0,NaN,700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,6000.0,4060.50,32760.00,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,450.79,7727.42,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,34848.00,180267.97,105623.29,58076.62,13000.00,NaN
5,NaN,NaN,NaN,24000.0,22600.00,7593.00,99305.00,125616.42,85019.22,72305.20,137182.55,52514.59
6,NaN,NaN,NaN,3000.0,8000.00,6780.00,1500.00,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,59513.00,79655.00,54673.13,56079.99,222056.54,93094.83
8,NaN,NaN,NaN,NaN,NaN,4987.00,46538.00,NaN,NaN,NaN,NaN,NaN
9,NaN,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
writer = pd.ExcelWriter(''+ruta_guardar+'/Gestores/Credifiel/Puntos comparativa '+now+'.xlsx',engine='xlsxwriter')
gestor_mes2.to_excel(writer,'BSC',index=False,header=True)
control_performance_cred_turno.to_excel(writer,'Turno',index=False,header=True)
control_performance_cred2.to_excel(writer,'Gestor',index=False,header=True)


writer.save()